In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import LongformerTokenizer, LongformerForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

2025-08-26 04:17:20.677348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756181840.937825      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756181841.028035      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
login(token=hf_token)

In [3]:
# Load your dataset
training_df = pd.read_csv("/kaggle/input/bcttrainingdata/BCTTrainingData.csv")  # columns: "text", "Patient Type"

# Define all possible labels
label_list = ["DCIS", "Stage I", "Stage II", "Stage III", "Stage IV", "Healthy", "High-Risk"]

# Encode labels as multi-hot vectors
def encode_labels(label_str):
    labels = [0]*len(label_list)
    for stage in label_str.split("; "):
        stage = stage.strip()
        if stage in label_list:
            labels[label_list.index(stage)] = 1
    return labels

training_df["encoded_labels"] = training_df["Patient Type"].apply(encode_labels)

# Split train / validation
train_df = training_df.sample(frac=0.8, random_state=42)
val_df = training_df.drop(train_df.index)

In [4]:
class CancerStageDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=4096):
        self.df = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx]["raw_text"]
        labels = torch.tensor(self.df.iloc[idx]["encoded_labels"], dtype=torch.float)
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        item = {key: val.squeeze() for key, val in encoding.items()}
        item["labels"] = labels  # as to be labels
        return item

# Starting the tokenizer
tokenizer = LongformerTokenizer.from_pretrained("yikuan8/Clinical-Longformer")

# Lastly is creating the datasets
train_dataset = CancerStageDataset(train_df, tokenizer)
val_dataset = CancerStageDataset(val_df, tokenizer)

tokenizer_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [5]:
model = LongformerForSequenceClassification.from_pretrained(
    "yikuan8/Clinical-Longformer",
    num_labels=len(label_list),
    problem_type="multi_label_classification"
)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/595M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at yikuan8/Clinical-Longformer and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",   # instead of evaluation_strategy
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    report_to="none"
)

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [8]:
trainer.train()

Initializing global attention on CLS token...


model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,0.264500,0.318773
2,0.296000,0.261724
3,0.308300,0.251425


TrainOutput(global_step=9186, training_loss=0.27198704478269875, metrics={'train_runtime': 9856.7229, 'train_samples_per_second': 0.932, 'train_steps_per_second': 0.932, 'total_flos': 2.413622364689203e+16, 'train_loss': 0.27198704478269875, 'epoch': 3.0})

In [9]:
# Predict on validation set
preds_output = trainer.predict(val_dataset)
pred_probs = torch.sigmoid(torch.tensor(preds_output.predictions)).numpy()

# Threshold predictions at 0.5
y_pred = (pred_probs >= 0.5).astype(int)
y_true = torch.stack([val_dataset[i]["labels"] for i in range(len(val_dataset))]).numpy()

# Print full multi-label classification report
print(classification_report(y_true, y_pred, target_names=label_list))

              precision    recall  f1-score   support

        DCIS       0.65      0.28      0.39        85
     Stage I       0.87      0.93      0.90       302
    Stage II       0.92      0.94      0.93       323
   Stage III       0.86      0.95      0.90       521
    Stage IV       0.95      0.94      0.94       444
     Healthy       0.81      0.32      0.46        41
   High-Risk       0.91      0.53      0.67        40

   micro avg       0.89      0.89      0.89      1756
   macro avg       0.85      0.70      0.74      1756
weighted avg       0.89      0.89      0.88      1756
 samples avg       0.90      0.92      0.89      1756



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
# Pick a repo name
repo_name = "bct-prototype-longformer-model"

# Push model + tokenizer
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/595M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Xpsloan/bct-prototype-longformer-model/commit/1c9a3e42cdfd7d2618d59882e393b07f13d33f66', commit_message='Upload tokenizer', commit_description='', oid='1c9a3e42cdfd7d2618d59882e393b07f13d33f66', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Xpsloan/bct-prototype-longformer-model', endpoint='https://huggingface.co', repo_type='model', repo_id='Xpsloan/bct-prototype-longformer-model'), pr_revision=None, pr_num=None)